# UAV Semantic Segmentation

## Import Library

In [7]:
import zipfile
import os
import shutil
from PIL import Image

## Data Collection

Data that will be used comes from kaggle, the dataset is available to download at https://www.kaggle.com/datasets/humansintheloop/semantic-segmentation-of-aerial-imagery (also available in this repo as Semantic segmentation dataset.zip).

## Preprocessing Data

In [8]:
# Path
zip_path = './Semantic segmentation dataset.zip'
output_base_path = './dataset'

In [15]:
# Function to resize images to height 420px while keeping aspect ratio and save with new names
def process_and_rename(input_folder, output_folder, start_count, prefix):
    count = start_count
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            # Open image
            img = Image.open(file_path)

            # Convert to RGB if the image is in 'P' or other modes
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
            # Resize keeping aspect ratio
            new_height = 420
            width_percent = new_height / float(img.size[1])
            new_width = int((float(img.size[0]) * float(width_percent)))
            img_resized = img.resize((new_width, new_height), Image.ANTIALIAS)
            
            # Save as .jpg with a sequential name
            output_path = os.path.join(output_folder, f"{prefix}_{count:03d}.jpg")
            img_resized.save(output_path, 'JPEG')
            
            count += 1
    return count

In [16]:
# Function to extract images and masks, and save them to new directories
def process_zip(zip_path, output_base_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_base_path)
    
    # Walking through the extracted folder to find images and masks
    global tile_folders 
    tile_folders = [f for f in os.listdir(output_base_path) if os.path.isdir(os.path.join(output_base_path, f))]

    # Output directories for images and masks
    images_output_folder = os.path.join(output_base_path, "images")
    masks_output_folder = os.path.join(output_base_path, "masks")
    
    os.makedirs(images_output_folder, exist_ok=True)
    os.makedirs(masks_output_folder, exist_ok=True)
    
    count = 1
    for tile_folder in tile_folders:
        tile_path = os.path.join(output_base_path, tile_folder)
        
        images_folder = os.path.join(tile_path, "images")
        masks_folder = os.path.join(tile_path, "masks")
        
        if not os.path.exists(images_folder) or not os.path.exists(masks_folder):
            continue
        
        # Processing images and masks, saving them with sequential names
        process_and_rename(images_folder, images_output_folder, count, prefix="image")
        count = process_and_rename(masks_folder, masks_output_folder, count, prefix="mask")

In [17]:
# Running the function
process_zip(zip_path, output_base_path)

C:\Users\Radit1812\AppData\Local\Temp\ipykernel_25204\1818994217.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((new_width, new_height), Image.ANTIALIAS)


In [18]:
for folder in tile_folders:
    shutil.rmtree(os.path.join(output_base_path, folder))

os.remove(os.path.join(output_base_path, "classes.json"))

## Conclusion

The preprocessing results in unziping dataset, compile them into folder *images* and *masks*, **masks** acts as a **ground truth file**. The total sets are **72 images**